In [90]:
from generate_patient_dataset import GeneratePatientDataset
data = GeneratePatientDataset("207")

readings = data.readings
annotations = data.annotations
mapping = data.mapping

In [91]:
sequences_mapping = [list(pair) for pair in mapping.keys()]
labels = [annotation.annotation for annotation in mapping.values()]

In [92]:
sequences = []
for sequence in sequences_mapping:
    entries = []
    for entry in sequence:
        entries.append([entry.ml_ii, entry.v_1])
    sequences.append(entries)

In [94]:
# Get max. length for Padding
max_length = 0

for key, val in mapping.items():
    current_length = len(key)
    if current_length > max_length:
        max_length = current_length

print("Maximum sequence length:", max_length)

Maximum sequence length: 924


In [ ]:
import pandas as pd

# Load the annotations data
annotations_path = '/mnt/data/207annotations.csv'
annotations = pd.read_csv(annotations_path)

# Convert the time column to minutes
annotations['Time_min'] = annotations['Time'] / 60

# Define the intervals for analysis
intervals = [(0, 7.5), (7.5, 15), (15, 22.5), (22.5, 30)]

# Focus on the specified classes
classes_of_interest = ['L', '!', 'A', 'V', 'E']

# Function to count occurrences of classes in each interval
def count_classes_in_interval(df, interval, classes):
    start, end = interval
    interval_data = df[(df['Time_min'] >= start) & (df['Time_min'] < end)]
    return interval_data['Type'].value_counts().reindex(classes, fill_value=0)

# Count occurrences for each interval
class_counts = {f'{start}-{end} min': count_classes_in_interval(annotations, (start, end), classes_of_interest)
                for start, end in intervals}

class_counts_df = pd.DataFrame(class_counts)
class_counts_df


In [95]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Pre-Processing
def preprocess_sequence(sequences, labels, max_length):
    # Convert labels to integer indices
    label_to_index = {'L': 0, 'A': 1, 'V': 2, '!': 3, 'E': 4}
    indices = [label_to_index[label] for label in labels]

    # Padding
    padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post', truncating='post', value=-1)

    # Frequency Bandpass Filter
    # TODO
    
    return padded_sequences, indices


In [ ]:
preprocessed_sequences = preprocess_sequence(sequences, labels, max_length)

In [100]:
from sklearn.model_selection import train_test_split

# Split data into train/test
train_x, test_x, train_y, test_y = train_test_split(
    sequences, labels, test_size=0.5, random_state=42, stratify=labels
)

In [109]:
import numpy as np

# Check for data balance between train test
train_class_distribution = np.bincount(train_y)
test_class_distribution = np.bincount(test_y)

print("Training Set Class Distribution:", train_class_distribution)
print("Testing Set Class Distribution:", test_class_distribution)

Training Set Class Distribution: [728  53  53 236  53]
Testing Set Class Distribution: [729  54  52 236  52]
